In [1]:
import pandas as pd
import numpy as np
import os
import cv2
import shutil
import matplotlib.image as img

In [2]:
def replacer(string):
    result = ""
    for char in string:
        if char == "-":
            result += " "
        else:
            result += char
    return result[12:-15]

In [3]:
def brightest(matrix):
    mini = 0
    mini_h = 0
    mini_v = 0
    mini_e = 0
    for i in range((matrix).shape[0]):
        for y in range((matrix).shape[1]):
            if (matrix[i,y] > mini) or (matrix[i,y] == mini and i*i + y*y < mini_e):
                mini = matrix[i,y]
                mini_h = i
                mini_v = y
                mini_e = i*i + y*y #no need to use sqrt
    return mini_h, mini_v                

In [4]:
col = ['Id', 'Filename', 'Description', 'Width', 'Height', 'Mean color',
         'Median brightness', 'Horizontal idx', 'Vertical idx']
df = pd.DataFrame(columns = col)

In [5]:
src = "C:/Users/Olaf/Desktop/Rekrutacja DS/images/"
for i, file in enumerate(os.listdir(src)):
    data = {}
    image = img.imread(src+file)
    image_g = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    data['Id'] = file[-14:-4]
    data['Filename'] = file
    data['Description'] = replacer(file)
    data['Width'] = image.shape[1]
    data['Height'] = image.shape[0]
    data['Mean color'] = [round(image[:,:,0].mean(),2), round(image[:,:,1].mean(),2), round(image[:,:,2].mean(),2)]
    data['Median brightness'] = np.median(image_g)
    data['Horizontal idx'], data['Vertical idx'] = brightest(image_g)
    df = df.append(data, ignore_index=True)

In [6]:
df.to_csv('images.csv')

In [7]:
medians = {}
for file in os.listdir(src):
    image = img.imread(src+file)
    image_g = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    medians[file] = np.median(image_g)

In [8]:
os.mkdir('agg-images')

In [10]:
for i, file in enumerate(sorted(medians, key=medians.__getitem__)):
    if i % 4 == 0:
        os.mkdir('agg-images/['+str(i//4+1)+']-images')
    shutil.move(src+file,'agg-images/['+str(i//4+1)+']-images/'+file)